# Setup

In [25]:
!pip install -qqq pytorch-lightning

In [26]:
#the tutorial code fails if this code is not present because an incompatibility between tensorboard installed with tensorflow by default in the environment and the tensorboard installed with pytorch
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [48]:
import pytorch_lightning as pl

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class NNet(pl.LightningModule):
  def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(1, 6, 5)
      self.pool = nn.MaxPool2d(2, 2)
      self.conv2 = nn.Conv2d(6, 16, 5)
      self.fc1 = nn.Linear(16 * 4 * 4, 120)
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)

      

  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1, 16 * 4 * 4)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

  def configure_optimizers(self):
    return optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
    
  
  def training_step(self, batch, batch_idx):
    # get the inputs; data is a list of [inputs, labels]
    x, y = batch
    logits = self(x)
    criterion = nn.CrossEntropyLoss()
    loss = criterion(logits, y)
    self.log('train_losss', loss)
    return loss




In [49]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [52]:
from pytorch_lightning import Trainer

nnet = NNet()
trainer = Trainer(gpus=1)
trainer.fit(nnet, trainloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | conv1 | Conv2d    | 156   
1 | pool  | MaxPool2d | 0     
2 | conv2 | Conv2d    | 2.4 K 
3 | fc1   | Linear    | 30.8 K
4 | fc2   | Linear    | 10.2 K
5 | fc3   | Linear    | 850   
------------------------------------
44.4 K    Trainable params
0         Non-trainable params
44.4 K    Total params
0.178     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


#Execution

In [51]:
!pip install -qqq pyngrok

from getpass import getpass
from pyngrok import ngrok

AUTH_TOKEN = getpass('Enter the NGROK auth token: ')
PORT = 6006

## Expose using ngrok
if AUTH_TOKEN:
    ngrok.set_auth_token(AUTH_TOKEN)

active_tunnels = ngrok.get_tunnels()
for tunnel in active_tunnels:
    public_url = tunnel.public_url
    ngrok.disconnect(public_url)

url = ngrok.connect(addr=PORT, bind_tls=True)

print(f"Tensorboard can be accessed on: {url}")

Enter the NGROK auth token: ··········
Tensorboard can be accessed on: NgrokTunnel: "https://a058-35-196-146-196.ngrok.io" -> "http://localhost:6006"
